# Question Answering – Fine-tuning – DistilBERT-base


Ce notebook entraîne un modèle de Question Answering extractif
sur les données SQuAD préprocessées.



## Objectifs

- Charger les données préprocessées
- Charger un modèle QA pré-entraîné
- Fine-tuner avec Trainer
- Sauvegarder le meilleur modèle


## Pourquoi DistilBERT ?

DistilBERT est une version compressée de BERT, obtenue par distillation de connaissances.
Il contient moins de paramètres (40 % de réduction), ce qui permet un temps d'entraînement
et d'inférence plus rapide, au prix d'une légère baisse de performance attendue.
Cela le rend idéal pour une comparaison équitable avec BERT et RoBERTa.

In [1]:
from datasets import load_from_disk
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    DefaultDataCollator
)
import numpy as np


c:\Users\aissi\OneDrive - De Vinci\A5\UVSQ\S2\Fouille de données\qa-project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from transformers import set_seed

set_seed(42)

In [2]:
import torch

print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Nombre de GPUs: {torch.cuda.device_count()}")
else:
    print("Pas de GPU disponible - utilisation du CPU")

CUDA disponible: True
GPU: NVIDIA GeForce RTX 3060 Laptop GPU
Nombre de GPUs: 1


## Chargement des données préprocessées

Les données ont été prétraitées dans le notebook précédent et sont chargées directement 
depuis le disque. On utilise une version réduite du dataset SQuAD afin de limiter le 
temps de calcul tout en conservant la structure du problème de Question Answering.

In [3]:
tokenized_datasets = load_from_disk("outputs/tokenized_squad_small")

## Data collator

Permet de créer les batchs correctement.


In [4]:
data_collator = DefaultDataCollator()


## Tokenizer

Le tokenizer associé au modèle est chargé pour assurer la cohérence
du pipeline d’entraînement et d’inférence.


In [5]:
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


## Modèle

On charge un modèle compatible Question Answering.


In [6]:
model_checkpoint = "distilbert-base-uncased"
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

c:\Users\aissi\OneDrive - De Vinci\A5\UVSQ\S2\Fouille de données\qa-project\.venv\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aissi\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading weights: 100%|██████████| 100/100 [00:00<00:00, 294.17it

## Paramètres d’entraînement


Grâce au nombre réduit de paramètres de DistilBERT, un batch size plus élevé (16) 
peut être utilisé sans dépasser les limites mémoire du GPU, optimisant ainsi l'utilisation
des ressources disponibles.

In [ ]:
training_args = TrainingArguments(
    output_dir="outputs/checkpoints/distilbert",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="outputs/logs/distilbert",
    logging_steps=50,
    save_total_limit=1,
    load_best_model_at_end=True
)

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


## Entraînement

On lance le fine-tuning avec Trainer.


In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator
)

In [9]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,3.958214,3.812345


Writing model shards: 100%|██████████| 1/1 [00:00<00:00,  1.74it/s]
There were missing keys in the checkpoint model loaded: ['distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias'].
There were unexpected keys in the checkpoint model loaded: ['distilbert.embeddings.LayerNorm.beta', 'distilbert.embeddings.LayerNorm.gamma'].


TrainOutput(global_step=125, training_loss=4.292656433105469, metrics={'train_runtime': 217.2468, 'train_samples_per_second': 9.206, 'train_steps_per_second': 0.575, 'total_flos': 195979650048000.0, 'train_loss': 4.292656433105469, 'epoch': 1.0})

In [ ]:
trainer.state.log_history

## Sauvegarde

In [10]:
trainer.save_model("outputs/checkpoints/distilbert/final")
tokenizer.save_pretrained("outputs/checkpoints/distilbert/final")


Writing model shards: 100%|██████████| 1/1 [00:00<00:00,  1.32it/s]


('outputs/checkpoints/distilbert/final\\tokenizer_config.json',
 'outputs/checkpoints/distilbert/final\\tokenizer.json')

## Conclusion

Le modèle a été fine-tuné et sauvegardé.
La prochaine étape consistera à évaluer la qualité (EM/F1) et la latence,
puis à comparer 3 modèles.
